# Finetuned GPT-4o-mini

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00


In [ ]:
import time
import json
import os
from openai import OpenAI

In [ ]:
# Only use this code block if you are using Google Colab.
# If you are using Jupyter Notebook, please ignore this code block. You can directly upload the file to your Jupyter Notebook file systems.
from google.colab import files

## It will prompt you to select a local file. Click on “Choose Files” then select and upload the file.
## Wait for the file to be 100% uploaded. You should see the name of the file once Colab has uploaded it.
uploaded = files.upload()

In [ ]:
def str_to_bool(string):
  if string == "True":
    return True
  elif string == "False":
    return False
  return False

In [ ]:
test_set = []
y_test = []

with open("test_set.jsonl", "r") as file:
  for line in file:
    data = json.loads(line)
    test_set.append(data['messages'])
    y_test.append(str_to_bool(data['messages'][2]['content']))

In [ ]:
api_key = "" # TODO
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
client = OpenAI()
our_model = "gpt-4o-mini-2024-07-18"

## Finetuning

In [ ]:
response = client.files.create(
  file=open("training_set.jsonl", "rb"),
  purpose="fine-tune"
)

In [ ]:
response

FileObject(id='file-0qAO84KW3kKMdmGsgN5piXUD', bytes=6882, created_at=1729887705, filename='training_set.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
file_id = response.id
file_id

'file-0qAO84KW3kKMdmGsgN5piXUD'

In [ ]:
response = client.fine_tuning.jobs.create(
  training_file=file_id,
  model=our_model
)

In [ ]:
fine_tuning_job_id = response.id
fine_tuning_job_id

'ftjob-rz6LNqLkGLocoIX9jVYiwAAI'

In [ ]:
status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
status

FineTuningJob(id='ftjob-rz6LNqLkGLocoIX9jVYiwAAI', created_at=1729887707, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=[], seed=502400017, status='validating_files', trained_tokens=None, training_file='file-0qAO84KW3kKMdmGsgN5piXUD', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

## Run the Finetuned Model

**Do not run the following codes until the fine-tune work ends. Check whether it ends in your [fine-tuning UI](https://platform.openai.com/finetune/).**

In [ ]:
status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
status

FineTuningJob(id='ftjob-rz6LNqLkGLocoIX9jVYiwAAI', created_at=1729887707, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AMLFsZg8', finished_at=1729888282, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-QkRPg4EtMRRKCnk01F7FjPky'], seed=502400017, status='succeeded', trained_tokens=15160, training_file='file-0qAO84KW3kKMdmGsgN5piXUD', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
fine_tuned_model = status.fine_tuned_model
fine_tuned_model

'ft:gpt-4o-mini-2024-07-18:personal::AMLFsZg8'

In [ ]:
y_pred = []
start = time.time()

for i in range(len(test_set)):
  completion = client.chat.completions.create(
    model=fine_tuned_model,
    messages=test_set[i]
  )
  y_pred.append(str_to_bool(completion.choices[0].message.content))

end = time.time()
print(f"{end - start:.4f} seconds")

2.7519 seconds


## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
